In [1]:
import os
os.chdir('..')

In [2]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [8]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
from model.CVAE_lstm_decoder import CVAE
from decode import *
import matplotlib.pyplot as plt

In [4]:
melody = np.load('./data/melody_baseline.npy')
chord = np.load('./data/number_96.npy')
chord_onehot = np.load('./data/onehot_96.npy')
length = np.load('./data/length.npy')

In [6]:
val_size = 500
print('splitting testing set...')
val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_melody = torch.from_numpy(melody[:val_size]).float()
val_length = torch.from_numpy(length[:val_size])

splitting testing set...


In [9]:
# Load model
device = 'cpu'
print('building model...')
model = CVAE(device = device).to(device)
model.load_state_dict(torch.load('output_models/model_weight_cvae_lstm_decoder.pth'))

model.eval()
chord, length, melody = val_chord.to(device), val_length.to(device), val_melody.to(device)

building model...


In [166]:
# Sampling
i = 80
sample_melody = melody[i].unsqueeze(0)
sample_length = val_length[i]
print(sample_length)

latent_size = 16

for k in range(10):
    latent = torch.rand(1,272,latent_size)
    z = torch.cat((latent,sample_melody), dim=-1)
    output, chord_pred = model.decode(z)

    gen_chord_index = torch.max(chord_pred[:,:sample_length],2).indices
    print(gen_chord_index)
#     print(gen_chord_index.shape)



tensor(9)
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 73, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56, 60]])
tensor([[ 0, 76, 73, 73, 40, 40, 60, 56,  0]])


In [155]:
sample_melody.shape

torch.Size([1, 272, 576])